In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommender').getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
df = spark.read.csv('data/movielens_ratings.csv', inferSchema=True, header=True)

In [4]:
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
training, test = df.randomSplit([0.8, 0.2])

model = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating').fit(training)

In [9]:
predictions = model.transform(test)
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    19|  0.7274857|
|     31|   2.0|    25|  1.9589896|
|     31|   1.0|     0|   3.712226|
|     85|   1.0|    26|  2.0322452|
|     85|   4.0|     7|  1.9257421|
|     85|   1.0|    29| 0.81892496|
|     65|   1.0|    28|-0.26452273|
|     65|   2.0|     3|  1.3543811|
|     65|   1.0|    19|  1.6801325|
|     65|   1.0|     4| 0.45138067|
|     65|   1.0|    24| 0.43564254|
|     65|   1.0|     2|   1.968709|
|     53|   1.0|    12|  1.7940761|
|     53|   1.0|     6|   2.637445|
|     78|   1.0|    12|  1.3601123|
|     78|   1.0|    20|  1.3448105|
|     78|   1.0|     8| 0.53170186|
|     78|   1.0|    11| 0.37368804|
|     34|   1.0|    14| -1.7396798|
|     81|   1.0|    22|  3.2635655|
+-------+------+------+-----------+
only showing top 20 rows



In [10]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
rmse = evaluator.evaluate(predictions)
print('RMSE:', rmse)

RMSE: 1.859233159522903


In [11]:
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     10|    11|
|     12|    11|
|     35|    11|
|     64|    11|
|     67|    11|
|     78|    11|
|     80|    11|
|     88|    11|
|     90|    11|
+-------+------+



In [13]:
recommendations = model.transform(single_user)
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     10|    11|   5.083405|
|     88|    11|   2.031008|
|      0|    11|  1.5923538|
|     12|    11|  1.3154404|
|     80|    11|   1.184302|
|     90|    11|  0.6311766|
|     78|    11| 0.37368804|
|     67|    11| 0.09147684|
|     64|    11|0.005408719|
|     35|    11|-0.82520574|
+-------+------+-----------+

